<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

# Create Initial Dataframe with Toronto Postal Codes, Boroughs, and Neighborhoods
Scrape Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M in order to obtain the data that is in the table of postal codes and transform the data into a pandas dataframe.

Download Dependencies

In [57]:
import pandas as pd # library for data analysis

# !conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

import requests # library to handle requests

Read the wiki page into a file and parse it.

In [58]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_file = requests.get(url).text
soup = BeautifulSoup(html_file, 'lxml')

Drill down into the table to extract headings and rows, then create DataFrame with headings.

In [59]:
table = soup.find('table', class_='wikitable sortable') # Get the Postcode/Borough/Neighbourgood table.
headings = table.find_all('th') # Extract the 3 column headings.
rows = table.find_all('tr') # Get all rows in the table
# Create a DataFrame with the column headings, removing the newline character from the 3rd heading.
df = pd.DataFrame(columns=[headings[0].text, headings[1].text, headings[2].text.split('\n')[0]])

Loop through all rows to build the DataFrame one row at a time.

In [60]:
for row in rows[1:]: # Skip the 1st (header) row.
    # Get the Postcode, Borough, and Neighbourhood for the current row, removing the trailing newline from the neighborhood.
    columns = row.find_all('td')
    postcode = columns[0].text
    borough = columns[1].text
    neighborhood = columns[2].text.split('\n')[0]
    if borough != 'Not assigned': # Skip any rows without a borough.
        if neighborhood == 'Not assigned': # Unassigned neighborhoods take on the name of their borough.
            neighborhood = borough
        if postcode in df['Postcode'].values:
            # Group neighboorhoods within same postcode into single postcode row.
            # Assumption: A postcode includes only one borough (though boroughs may span multiple postcodes).
            df.loc[df['Postcode'] == postcode, 'Neighbourhood'] = df[df['Postcode'] == postcode]['Neighbourhood'] + ", " + neighborhood
        else: # Add row for new postcode, borough, and neighborhood.
            df = df.append(pd.Series([postcode, borough, neighborhood], index=df.columns), ignore_index=True)
df.head() # Display first 5 rows of DataFrame

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [61]:
df.shape

(103, 3)